In [1]:
import random
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

def generate_truck_availability(truck_numbers):
    return {truck: random.choice([0, 1]) for truck in truck_numbers}
def load_input_data_from_csv(file_path):
    input_data = pd.read_csv(file_path)
    relevant_fuel_parameters = ['HSD_Requirement', 'LDO_Requirement', 'FO_Requirement', 
                                'LSHS_Requirement', 'SKO_Requirement', 'MS_Requirement']
    input_data = input_data[relevant_fuel_parameters].to_dict(orient='records')[0]
    return input_data

In [2]:
data = pd.read_csv("Combined_Dataset.csv")
data2 = data.iloc[:18000].copy() 
truck_numbers = data2['Truck_Number'].unique()
truck_availability_status = generate_truck_availability(truck_numbers)
data2['Truck_Availability'] = data2['Truck_Number'].map(truck_availability_status)
data2 = data2[data2['Truck_Availability'] == 1].copy()  

# relevant_columns = ['HSD_Requirement', 'LDO_Requirement', 'FO_Requirement', 
#                     'LSHS_Requirement', 'SKO_Requirement', 'MS_Requirement', 
#                     'Truck_Capacity', 'Weather', 'Traffic', 'Local_Infrastructure', 
#                     'Night_Driving', 'Customer_Satisfaction', 'Truck_Number']
relevant_columns = ['HSD_Requirement', 'LDO_Requirement', 'FO_Requirement', 
                    'LSHS_Requirement', 'SKO_Requirement', 'MS_Requirement', 
                    'Truck_Capacity', 'Weather', 'Traffic', 'Local_Infrastructure', 
                    'Night_Driving', 'Truck_Number']

In [3]:
truck_availability_status

{'Cluster1_Truck14': 1,
 'Cluster2_Truck25': 1,
 'Cluster3_Truck42': 1,
 'Cluster4_Truck68': 0,
 'Cluster5_Truck87': 1,
 'Cluster6_Truck101': 0,
 'Cluster7_Truck123': 1,
 'Cluster8_Truck159': 0,
 'Cluster9_Truck170': 0,
 'Cluster10_Truck197': 1,
 'Cluster11_Truck211': 0,
 'Cluster12_Truck232': 0,
 'Cluster13_Truck259': 0,
 'Cluster14_Truck265': 1,
 'Cluster15_Truck296': 0,
 'Cluster16_Truck302': 1,
 'Cluster17_Truck331': 0,
 'Cluster18_Truck341': 0,
 'Cluster19_Truck369': 1,
 'Cluster20_Truck391': 1,
 'Cluster21_Truck414': 0,
 'Cluster22_Truck427': 0,
 'Cluster23_Truck445': 0,
 'Cluster24_Truck476': 1,
 'Cluster25_Truck496': 0,
 'Cluster26_Truck501': 1,
 'Cluster27_Truck535': 0,
 'Cluster28_Truck550': 0,
 'Cluster29_Truck567': 0,
 'Cluster30_Truck586': 0,
 'Cluster31_Truck616': 1,
 'Cluster32_Truck621': 1,
 'Cluster33_Truck641': 0,
 'Cluster34_Truck679': 1,
 'Cluster35_Truck693': 1,
 'Cluster36_Truck704': 1,
 'Cluster37_Truck734': 0,
 'Cluster38_Truck758': 1,
 'Cluster39_Truck763': 1,


In [4]:
truck_availability_df = pd.DataFrame(truck_availability_status.items(), columns=['Truck_Number', 'Truck_Availability'])
truck_availability_df.to_excel('Real_Time_Truck_Availability_Status.xlsx', index=False)

In [5]:
clusters = data2['Cluster_ID'].unique()
models = {}
for cluster in clusters:
    cluster_data = data2[data2['Cluster_ID'] == cluster]
    if len(cluster_data) == 0:  
        continue
    cluster_data = cluster_data[relevant_columns]  
    label_encoders = {}
    #for column in ['Weather', 'Traffic', 'Local_Infrastructure', 'Night_Driving', 'Customer_Satisfaction']:
    for column in ['Weather', 'Traffic', 'Local_Infrastructure', 'Night_Driving']:    
        label_encoders[column] = LabelEncoder()
        cluster_data[column] = label_encoders[column].fit_transform(cluster_data[column])
    X = cluster_data.drop(columns=['Truck_Number'])
    y = cluster_data['Truck_Number']
    model = RandomForestClassifier()
    model.fit(X, y)
    models[cluster] = model

def predict_truck_allocation(input_data, cluster_id):
    input_df = pd.DataFrame([input_data])
   # for column in ['Weather', 'Traffic', 'Local_Infrastructure', 'Night_Driving', 'Customer_Satisfaction']:
    for column in ['Weather', 'Traffic', 'Local_Infrastructure', 'Night_Driving']:    
        input_df[column] = label_encoders[column].transform([input_data[column]])   
    model = models.get(cluster_id)
    if model is None:
        return "No Truck is Currently Available"    
    predicted_truck = model.predict(input_df.drop(columns=['Customer_ID']))
    return predicted_truck[0]  

def find_cluster_id(input_data):
    customer_id = input_data['Customer_ID']
    customer_data = data[data['Customer_ID'] == customer_id]
    if len(customer_data) == 0:
        print("Customer ID not found in the dataset")
        return None
    cluster_id = customer_data.iloc[0]['Cluster_ID']
    return cluster_id

In [7]:
input_data = load_input_data_from_csv("Final_Fuel_Requirement_Predictions.csv")
input_data.update({
    'Truck_Capacity': 250,
    'Weather': 'Drizzle', 
    'Traffic': 3,  
    'Local_Infrastructure': 'Poor', 
    'Night_Driving': 'Yes', 
   # 'Customer_Satisfaction': 'Good',  
    'Customer_ID': 'Customer845',
})
cluster_id = find_cluster_id(input_data)
if cluster_id is not None:
    predicted_truck = predict_truck_allocation(input_data, cluster_id)
    print(f"Allocate Truck Number for Customer_ID '{input_data['Customer_ID']}' is '{predicted_truck}' ")

Allocate Truck Number for Customer_ID 'Customer845' is 'Cluster45_Truck887' 
